# Adding Internal Energy RHS Terms (Option A)

This example shows how to add arbitrary symbolic cooling/heating terms to the internal energy equation using the `Network.add_internal_energy_term` API. The terms are included in code generation and analytically differentiated in the Jacobian.

In [ ]:
from jaff.network import Network
from sympy import symbols, IndexedBase

# Load a small demo network
network = Network("../networks/test.dat", label="test_with_energy")

# Define symbols used in energy terms
nden = IndexedBase('nden')  # species number densities
tgas = symbols('tgas')      # gas temperature symbol (used in rate expressions)
eint = symbols('eint')      # internal energy density symbol (optional)

# Add cooling/heating terms to d(eint)/dt.
# These can be SymPy expressions or strings parsable by SymPy.
network.add_internal_energy_term('-1e-22*nden[0]*nden[1]*tgas**0.5')  # example cooling
network.add_internal_energy_term('2e-24*nden[2]*tgas')                 # example heating

# Inspect stored energy terms
network.energy_rhs_terms

## Generate C++ (Kokkos ODE) Code
We now generate the C++ ODE system using the Kokkos plugin. The internal energy equation and Jacobian will include the terms added above.

In [ ]:
import os, jaff
from jaff.plugins.kokkos_ode import plugin as kokkos_plugin

# Locate the Kokkos ODE template directory
template_dir = os.path.join(os.path.dirname(jaff.__file__), 'templates', 'kokkos_ode')
# Choose an output directory for generated files
build_dir = os.path.join(os.getcwd(), 'build-energy-option-a')
os.makedirs(build_dir, exist_ok=True)

# Generate code (Option A: energy terms already attached to the network)
kokkos_plugin.main(network, template_dir, path_build=build_dir)

print('Generated files in:', build_dir)
print('Listing:')
print('
'.join(sorted(os.listdir(build_dir))))